In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [11]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('../database/intents1.json').read()
intents = json.loads(data_file)

for intent in intents['type1']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [12]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-12-7fa144a2511d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.7, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('../models/model1.h5', hist)

print("model created")

Epoch 1/200
17/17 [==============================] - 0s 1ms/step - loss: 2.9905 - accuracy: 0.0241
Epoch 2/200
17/17 [==============================] - 0s 1ms/step - loss: 2.9306 - accuracy: 0.1205
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 2.8455 - accuracy: 0.1446
Epoch 4/200
17/17 [==============================] - 0s 881us/step - loss: 2.8647 - accuracy: 0.1446
Epoch 5/200
17/17 [==============================] - 0s 1ms/step - loss: 2.7684 - accuracy: 0.1446
Epoch 6/200
17/17 [==============================] - 0s 888us/step - loss: 2.6965 - accuracy: 0.2048
Epoch 7/200
17/17 [==============================] - 0s 923us/step - loss: 2.6342 - accuracy: 0.1928
Epoch 8/200
17/17 [==============================] - 0s 927us/step - loss: 2.5432 - accuracy: 0.2530
Epoch 9/200
17/17 [==============================] - 0s 943us/step - loss: 2.4677 - accuracy: 0.3494
Epoch 10/200
17/17 [==============================] - 0s 852us/step - loss: 2.4058 - accuracy: 0.34

17/17 [==============================] - 0s 1ms/step - loss: 0.1438 - accuracy: 0.9639
Epoch 83/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1539 - accuracy: 0.9639
Epoch 84/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2043 - accuracy: 0.9398
Epoch 85/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2560 - accuracy: 0.9036
Epoch 86/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2068 - accuracy: 0.9518
Epoch 87/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2529 - accuracy: 0.9036
Epoch 88/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2102 - accuracy: 0.9518
Epoch 89/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1379 - accuracy: 0.9880
Epoch 90/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0963 - accuracy: 0.9880
Epoch 91/200
17/17 [==============================] - 0s 1ms/step - loss: 0.2447 - accuracy: 0.9036
Epoch 92/200


17/17 [==============================] - 0s 1ms/step - loss: 0.0520 - accuracy: 1.0000
Epoch 164/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1285 - accuracy: 0.9639
Epoch 165/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1174 - accuracy: 0.9639
Epoch 166/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0817 - accuracy: 0.9759
Epoch 167/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0823 - accuracy: 0.9759
Epoch 168/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0696 - accuracy: 0.9880
Epoch 169/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0596 - accuracy: 0.9880
Epoch 170/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1316 - accuracy: 0.9880
Epoch 171/200
17/17 [==============================] - 0s 1ms/step - loss: 0.1019 - accuracy: 0.9759
Epoch 172/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0778 - accuracy: 1.0000
Epoc